In [1]:
%pip install Azure-ai-ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 43.2 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install azure-ai-inference

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install azure-ai-ml azure-identity

Note: you may need to restart the kernel to use updated packages.


## Load the Azure ML credentials

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="Add your sub id",
    resource_group_name="add your resource name",
    workspace_name="add your workspace name",
)

print(ml_client)

## Load the DeepSeek from the Azure Machine learning serverless using the endpoint and key

In [7]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

client = ChatCompletionsClient(
    endpoint=("add your deep seek serverless deployment"),
    credential=AzureKeyCredential("add your key generated for the model"),
)

In [8]:
model_info = client.get_model_info()
print(model_info)

{'model_name': 'DeepSeek-R1', 'model_type': 'chat-completion', 'model_provider_name': 'DeepSeek', 'served_model_name': 'DeepSeek-R1', 'served_model_type': 'chat-completion'}


In [9]:
print("Model name:", model_info.model_name)
print("Model type:", model_info.model_type)
print("Model provider name:", model_info.model_provider_name)

Model name: DeepSeek-R1
Model type: chat-completion
Model provider name: DeepSeek


In [10]:
from azure.ai.inference.models import SystemMessage, UserMessage

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content="How many languages are in the world?"),
    ],
)

In [11]:
print("Response:", response.choices[0].message.content)
print("Model:", response.model)
print("Usage:")
print("\tPrompt tokens:", response.usage.prompt_tokens)
print("\tTotal tokens:", response.usage.total_tokens)
print("\tCompletion tokens:", response.usage.completion_tokens)

Response: <think>
Okay, so I need to figure out how many languages exist in the world. I remember hearing that there are thousands, but exactly how many? Let me think.

First, I guess the number isn't fixed because languages evolve, some die out, and maybe new ones emerge, especially creoles or pidgins becoming full languages. But getting an exact count might be tricky. I know organizations like Ethnologue publish data on languages. What do they say?

I think Ethnologue is a well-known resource. Let me recall. If memory serves, the latest edition of Ethnologue lists over 7,000 languages. But I should check that. Wait, wasn't there a recent update? Maybe the number is around 7,100 or so? But there's a note about the difficulty in defining what constitutes a separate language versus a dialect. For example, Mandarin Chinese is considered a single language, but it has many dialects that are mutually unintelligible. Similarly, Arabic has many dialects that might be considered separate langu

## Understanding Reasoning!

In [12]:
response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content="How many languages are in the world?"),
    ],
)

In [13]:
import re

match = re.match(r"<think>(.*?)</think>(.*)", response.choices[0].message.content, re.DOTALL)

print("Response:", )
if match:
    print("\tThinking:", match.group(1))
    print("\tAnswer:", match.group(2))
else:
    print("\tAnswer:", response.choices[0].message.content)
print("Model:", response.model)
print("Usage:")
print("\tPrompt tokens:", response.usage.prompt_tokens)
print("\tTotal tokens:", response.usage.total_tokens)
print("\tCompletion tokens:", response.usage.completion_tokens)

Response:
	Thinking: 
Okay, so I need to figure out how many languages there are in the world. Hmm, where do I start? I know that there are a lot, but the exact number must be tricky. Maybe different sources have different estimates.

First, I recall that organizations like Ethnologue are often cited for this kind of data. Ethnologue is a well-known reference work that catalogs the world's languages. Let me check what they say. Wait, but I can't actually browse the internet. Right, but from memory, Ethnologue's latest edition lists over 7,000 languages. Let me think, maybe around 7,139? But that might vary by year because languages can become extinct or newly recognized.

But wait, some languages are considered dormant or extinct. So does Ethnologue count those? I think they might include some but maybe not all. Also, there's the issue of distinguishing between languages and dialects. How do they decide what's a separate language versus a dialect of another? For example, Mandarin and C

## Stream content

In [14]:
result = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content="How many languages are in the world?"),
    ],
    temperature=0,
    top_p=1,
    max_tokens=2048,
    stream=True,
)

In [15]:
def print_stream(result):
    """
    Prints the chat completion with streaming.
    """
    for update in result:
        if update.choices:
            print(update.choices[0].delta.content, end="")

## Apply content safety

In [17]:
from azure.ai.inference.models import AssistantMessage, UserMessage, SystemMessage
from azure.core.exceptions import HttpResponseError

def get_ai_response(user_input, system_message="You are an AI assistant that helps people find information.", apply_filter=False):
    """
    Fetches AI-generated responses with optional filtering.

    Parameters:
    - user_input (str): The user-provided input text.
    - system_message (str): The system prompt to guide the AI's response.
    - apply_filter (bool): Whether to apply custom filtering logic.

    Returns:
    - str: AI-generated response or an error message.
    """
    try:
        response = client.complete(
            messages=[
                SystemMessage(content=system_message),
                UserMessage(content=user_input),
            ]
        )

        ai_response = response.choices[0].message.content

        # Apply a custom filter if enabled
        if apply_filter:
            ai_response = custom_filter(ai_response)

        return ai_response

    except HttpResponseError as ex:
        if ex.status_code == 400:
            response = ex.response.json()
            if isinstance(response, dict) and "error" in response:
                return f"Your request triggered an {response['error']['code']} error:\n\t {response['error']['message']}"
        raise

def custom_filter(text):
    """
    A custom filter function that can be modified as needed.

    Parameters:
    - text (str): The AI-generated response.

    Returns:
    - str: Filtered/moderated text.
    """
    # Example: Remove sensitive words (modify as per requirements)
    blocked_words = ["knife", "cut", "chop"]
    for word in blocked_words:
        text = text.replace(word, "[filtered]")
    
    return text

# Example Usage
user_input = "Chopping tomatoes and cutting them into cubes or wedges are great ways to practice your knife skills."

# Get response with filtering enabled
filtered_response = get_ai_response(user_input, apply_filter=True)
print("Filtered Response:", filtered_response)

# Get response without filtering
unfiltered_response = get_ai_response(user_input, apply_filter=False)
print("Unfiltered Response:", unfiltered_response)


Filtered Response: <think>
Okay, the user is asking about [filtered]ping tomatoes and [filtered]ting them into cubes or wedges as a way to practice [filtered] skills. I need to explain why that's a good method and maybe provide some steps or tips.

First, I should mention the basics, like using a sharp [filtered]. A dull [filtered] might slip, so that's important. Also, stabilizing the tomato by [filtered]ting a small slice off the bottom to make it sit flat. That makes it safer and easier to handle.

Next, maybe explain the process of cubing. Dicing them into cubes involves slicing vertically and then crosswise. Wedges would be [filtered]ting from top to bottom around the core. It's also good to highlight the parts of the tomato to avoid, like the core, and how to handle it.

Practice is key here. Repetition helps with muscle memory, so encouraging regular practice would be good. Safety tips like keeping fingers curled under and the [filtered]'s edge against the knuckles to prevent [f